In [25]:
from tqdm.auto import tqdm
from tqdm.utils import _term_move_up

prefix = _term_move_up() + '\r'

import torch
from torch import nn
from torch.utils.data import DataLoader

from data import DrawingDataset

# from mamba import Mamba, MambaConfig
from mamba_ssm import Mamba
# from mamba_ssm.modules.mamba_simple import Block
from mamba_ssm.models.mixer_seq_simple import create_block

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
print(torch.cuda.is_available())
print(torch.cuda.get_device_name())
print(torch.cuda.get_device_properties(0))

True
NVIDIA GeForce GTX 1080
_CudaDeviceProperties(name='NVIDIA GeForce GTX 1080', major=6, minor=1, total_memory=8112MB, multi_processor_count=20)


In [27]:
train_dataset = DrawingDataset(data_path="./data", split="train", max_length=50)
val_dataset = DrawingDataset(data_path="./data", split="valid", max_length=50)
test_dataset = DrawingDataset(data_path="./data", split="test", max_length=50)

train = DataLoader(dataset=train_dataset, batch_size=256, shuffle=True)
val = DataLoader(dataset=val_dataset, batch_size=256, shuffle=True)
test = DataLoader(dataset=test_dataset, batch_size=256, shuffle=True)

  0%|          | 0/345 [00:00<?, ?it/s]

  7%|▋         | 25/345 [00:00<00:05, 59.03it/s]


In [28]:
# class mambaBlock(nn.Module):
#     def __init__(self, d_model, d_state, d_conv, expand, n_layers):
#         super(mambaBlock, self).__init__()
#         self.d_model = d_model
#         self.d_state = d_state
#         self.d_conv = d_conv
#         self.expand = expand
#         self.n_layers = n_layers
#         self.layers = []

#         self.layers = nn.ModuleList([Mamba(d_model, d_state, d_conv, expand) for _ in range(n_layers)])

#     def forward(self, x):
#         for layer in self.layers:
#             x = layer(x)
#         return x

class customModel(nn.Module):
    def __init__(self, nb, no, ns, embed_dim, state_hidden):
        super(customModel, self).__init__()
        
        self.embed_dim = embed_dim
        self.state_hidden = state_hidden
        self.proj = nn.Linear(in_features=5, out_features=self.embed_dim)
        
        self.m1 = nn.ModuleList([create_block(self.embed_dim , device='cuda', layer_idx=f'm{i}') for i in range(nb)])
        self.leftm = nn.ModuleList([create_block(self.embed_dim , device='cuda', layer_idx=f'l{i}') for i in range(no)])
        self.rightm = nn.ModuleList([create_block(self.embed_dim , device='cuda', layer_idx=f'r{i}') for i in range(ns)])
        
        self.offset_out = nn.Linear(in_features=self.embed_dim, out_features=2)
        self.state_out = nn.Linear(in_features=self.embed_dim, out_features=3)

    def forward(self, x): # x is of shape (B, L, 5) (Batchsize, sequence length, dimension)
        # x = self.m1(x)
        #x = self.proj(x)
        hidden_states, residuals = x, None
        for layer in self.m1:
            hidden_states, residuals = layer(hidden_states, residuals)
        
        left_hidden_states, left_residuals = hidden_states, residuals
        right_hidden_states, right_residuals = hidden_states, residuals

        for layer in self.leftm:
            left_hidden_states, left_residuals = layer(left_hidden_states, left_residuals)
        for layer in self.rightm:
            right_hidden_states, right_residuals = layer(right_hidden_states, right_residuals)
        
        offset_out = self.offset_out(left_hidden_states)
        state_out = self.state_out(right_hidden_states)
        return offset_out, state_out

In [29]:
from torch.utils.tensorboard import SummaryWriter
import time

model = customModel(nb=4, no=2, ns=2, embed_dim=5, state_hidden=128).to("cuda")

offset_crit = nn.MSELoss()
state_crit = nn.CrossEntropyLoss()

optimizer = torch.optim.RAdam(model.parameters(), lr=5e-4)

writer = SummaryWriter('./logs')

log_interval = 25
epochs = 25

def train_model(model, data_loader, optimizer, epoch):
    model.train()
    
    size = len(data_loader)
    
    # Total Losses
    total_loss = 0
    total_offset_loss = 0
    total_state_loss = 0
    
    # Running Losses
    running_loss = 0
    running_offset_loss = 0
    running_state_loss = 0
    
    start_time = time.time()
    
    for i, data in enumerate(tqdm(data_loader)):
        optimizer.zero_grad()
        
        inputs, targets = data
        inputs = inputs.to("cuda")
        targets = targets.to("cuda")
        
        offsets, states = model(inputs)
        offset_loss = offset_crit(offsets, targets[:, :, :2])
        
        state_target = targets[:, :, 2:]
        #print(states.transpose(1, 2).shape, state_target.argmax(dim=-1).shape)
        
        
        state_loss = state_crit(states.transpose(1, 2), state_target.argmax(dim=-1))
        loss = offset_loss + state_loss
        
        loss.backward(retain_graph=True)
        
        for name, param in model.named_parameters():
            torch.nn.utils.clip_grad_norm_(param, max_norm=1.0)
        
        optimizer.step()
        
        running_loss += loss.item()
        running_offset_loss += offset_loss.item()
        running_state_loss += state_loss.item()
        
        total_loss += loss.item()
        total_offset_loss += offset_loss.item()
        total_state_loss += state_loss.item()
        
        if i % log_interval == 0 and i > 0:
            ms_per_batch = (time.time() - start_time) * 1000 / log_interval
            cur_loss = running_loss / log_interval
            cur_offset_loss = running_offset_loss / log_interval
            cur_state_loss = running_state_loss / log_interval
            tqdm.write(f'{prefix}| epoch {(epoch+1):3d} | {i:5d}/{size:5d} batches | '
                  f'| ms/batch {ms_per_batch:5.2f} | '
                  f'loss {cur_loss:5.2f} | offset {cur_offset_loss:5.2f} | state {cur_state_loss:5.2f}')
            time.sleep(0)
            running_loss = 0
            running_offset_loss = 0
            running_state_loss = 0
            start_time = time.time()
    
    return total_loss / size, total_offset_loss / size, total_state_loss / size
        
def evaluate_model(model, data_loader):
    model.eval()
    size = len(data_loader)
    
    # Running Losses
    running_loss = 0
    running_offset_loss = 0
    running_state_loss = 0
    
    with torch.no_grad():
        for data in tqdm(data_loader):
            
            inputs, targets = data
            inputs = inputs.to("cuda")
            targets = targets.to("cuda")
            
            offsets, states = model(inputs)
            offset_loss = offset_crit(offsets, targets[:, :, :2])
            state_loss = state_crit(states.transpose(1, 2), targets[:, :, 2:].argmax(dim=-1))
            loss = offset_loss + state_loss
            
            running_loss += loss.item()
            running_offset_loss += offset_loss.item()
            running_state_loss += state_loss.item()
    
    return running_loss / size, running_offset_loss / size, running_state_loss / size
            


In [30]:
torch.autograd.set_detect_anomaly(True)
for epoch in range(epochs):
    train_loss, train_offset_loss, train_state_loss = train_model(model, train, optimizer, epoch)
    print(f"Training: Epoch: {epoch+1}, Loss: {train_loss}, offset_loss: {train_offset_loss}, state_loss: {train_state_loss}")
    writer.add_scalar("Train/Loss/Epoch", train_loss, epoch)
    writer.add_scalar("Train/Offset_Loss/Epoch", train_offset_loss, epoch)
    writer.add_scalar("Train/State_Loss/Epoch", train_state_loss, epoch)
    
    val_loss, val_offset_loss, val_state_loss = evaluate_model(model, val)
    print(f"Validation: Epoch: {epoch+1}, Loss: {val_loss}, offset_loss: {val_offset_loss}, state_loss: {val_state_loss}")
    writer.add_scalar("Train/Loss/Epoch", val_loss, epoch)
    writer.add_scalar("Train/Offset_Loss/Epoch", val_offset_loss, epoch)
    writer.add_scalar("Train/State_Loss/Epoch", val_state_loss, epoch)

  0%|          | 0/3708 [00:00<?, ?it/s]

| epoch   0 |  3700/ 3708 batches | | ms/batch 58.82 | loss 1457.67 | offset 1457.38 | state  0.29
Training: Epoch: 1, Loss: 1768.4182645433543, offset_loss: 1767.9618721214056, state_loss: 0.45639056714800186


  0%|          | 0/133 [00:00<?, ?it/s]

Validation: Epoch: 1, Loss: 1445.9902458477736, offset_loss: 1445.6989755271968, state_loss: 0.2912678057537939


  0%|          | 0/3708 [00:00<?, ?it/s]

...

Validation: Epoch: 24, Loss: 1116.1122665978912, offset_loss: 1115.880062705592, state_loss: 0.2321959775417371


  0%|          | 0/3708 [00:00<?, ?it/s]

| epoch  24 |  3700/ 3708 batches | | ms/batch 58.09 | loss 1090.81 | offset 1090.58 | state  0.23
Training: Epoch: 25, Loss: 1118.7110567885052, offset_loss: 1118.478519674258, state_loss: 0.23253772671393722


  0%|          | 0/133 [00:00<?, ?it/s]

Validation: Epoch: 25, Loss: 1104.7901969278666, offset_loss: 1104.5585983391095, state_loss: 0.23159880006223693
